In [2]:
import nexussdk as nexus
import neurom as nm
import matplotlib.pyplot as plt
import getpass
import requests
from neurom.check import neuron_checks, structural_checks

## Retrieve the morphology from Nexus

In [2]:
deployment = "https://bbp.epfl.ch/nexus/v1"
token = getpass.getpass()

In [6]:
nexus.config.set_environment(deployment)
nexus.config.set_token(token)
org = ""
project = ""

In [8]:
morphology_id = ""

In [1]:
def retrieve_morphology(morphology_id):
    response = nexus.resources.fetch(org, project, morphology_id)
    file_url = response['distribution']['contentUrl']
    filename = response['distribution']['name']
    attachment_headers = {}
    attachment_headers["Authorization"] ="Bearer {}".format(token)
    
    content = requests.get(file_url, headers=attachment_headers).content
    
    file = open(filename, 'wb')
    file.write(content)
    file.close()
    
    return nm.load_neuron(filename)
    

In [30]:
morph = retrieve_morphology(morphology_id)

## Perform morphometry on the morphology

In [83]:
def morphometry_analysis(morphology):
    has_axon = neuron_checks.has_axon(morph).status
    dangling = neuron_checks.has_no_dangling_branch(morph).info
    z_jumps = neuron_checks.has_no_jumps(morph).info
    fat_ends = neuron_checks.has_no_fat_ends(morph).info
    narrow_start = neuron_checks.has_no_narrow_start(morph).info
    nonzero_neurite_radii = neuron_checks.has_all_nonzero_neurite_radii(morph).status
    
    morphometry = {
        "has_axon": has_axon,
        "dangling": len(dangling),
        "z_jump": len(z_jumps),
        "fat_ends": len(fat_ends),
        "narrow_start": len(narrow_start),
        "nonzero_neurite_radius": nonzero_neurite_radii
    }
    
    return morphometry

In [84]:
morphometry_analysis(morph)

{'dangling': 4,
 'fat_ends': 0,
 'has_axon': False,
 'narrow_start': 0,
 'nonzero_neurite_radius': True,
 'z_jump': 0}